In [1]:
import sys, os, time
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import unary_union

### Initial Raw File Imports

In [2]:
p = r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Submission_15_12_2018'
q = r'Recon_December.xlsx'

Trips = pd.read_excel(os.path.join(p,q), sheet_name = 'Trips')
Passengers = pd.read_excel(os.path.join(p,q), sheet_name = 'Passengers')

RouteMaster = pd.read_excel(os.path.join(p,r'Route O-D_Master_21December2018.xlsx'))
RouteMaster = RouteMaster[["UNIQUE ROUTE ID's",'ORIGIN','DESTINATION']].set_index("UNIQUE ROUTE ID's")

### Match on useful fields from Trips frame

In [3]:
def convert(x):
    try:
        return (x.minute + x.hour * 60 + x.second / 60)
    except:
        pass

Passengers['TT_minutes'] = Passengers['Travel Time'].apply(lambda x: convert(x))

Trips = Trips.set_index('Trip ID')
Passengers = Passengers.set_index('Trip ID')
for i in ['Route Description','Start Coordinate', 'End Coordinate', 'Revenue', 'Start Time', 'Total Passengers', 'Distance']:
          Passengers['Trip %s' % i] = Trips[i]
Passengers = Passengers.reset_index()

### Match on useful fields from RouteMaster frame

In [4]:
Passengers = Passengers.set_index('Trip Route Description')
Passengers['Trip Origin'] = RouteMaster['ORIGIN']
Passengers['Trip Destination'] = RouteMaster['DESTINATION']
Passengers = Passengers.reset_index()

### Error Handling: Remove Passengers where travel time, fare or trip distance values are erroneous

In [5]:
Passengers = Passengers.loc[(Passengers['TT_minutes'] < 1000) & (Passengers['TT_minutes'] > 0)]
Passengers = Passengers.loc[(Passengers['Fare'] < 1000) & (Passengers['Fare'] > 0)]
Passengers = Passengers.loc[(Passengers['Trip Distance'] < 1000) & (Passengers['Trip Distance'] > 0)]

### Add fare per minute / unit distance

In [6]:
Passengers['Fare_per_minute'] = Passengers['Fare'] / Passengers['TT_minutes']
Passengers['Fare_per_mile'] = Passengers['Fare'] / Passengers['Trip Distance']

### Passengers: Add on ward details for boarding / alightning

In [7]:
shp_pth = r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Fwd__SAL\1_Residential_SAL'
ward_shp = gpd.read_file(os.path.join(shp_pth, 'wards_4326.shp'))
ward_shp = ward_shp[['SL_WARD_KE','SL_SUB_CNC','geometry']]

Passengers['P_ID'] = Passengers.index
Bind_P = Passengers[['P_ID','Boarding Location','Alighting Location']]
def convert_to_point(x):
    l = x.split(', ')
    return Point(float(l[0]), float(l[1]))
    
Bind_P['Boarding Point'] = Bind_P['Boarding Location'].apply(lambda x: convert_to_point(x))
Bind_P['Alighting Point'] = Bind_P['Alighting Location'].apply(lambda x: convert_to_point(x))

Q = gpd.GeoDataFrame(Bind_P, crs = ward_shp.crs, geometry = 'Boarding Point')
boarding_join = gpd.sjoin(Q, ward_shp, how = 'left')
boarding_join = boarding_join[['P_ID','SL_WARD_KE','SL_SUB_CNC']]
boarding_join.columns = ['P_ID','Board_WARD_KE','Board_SUB_CNC']
boarding_join = boarding_join.set_index('P_ID')

Q2 = gpd.GeoDataFrame(Bind_P, crs = ward_shp.crs, geometry = 'Alighting Point')
alighting_join = gpd.sjoin(Q2, ward_shp, how = 'left')
alighting_join = alighting_join[['P_ID','SL_WARD_KE','SL_SUB_CNC']]
alighting_join.columns = ['P_ID','Alight_WARD_KE','Alight_SUB_CNC']
alighting_join = alighting_join.set_index('P_ID')

Passengers = Passengers.set_index('P_ID')
Passengers['Alight_WARD_KE'] = alighting_join['Alight_WARD_KE']
Passengers['Alight_SUB_CNC'] = alighting_join['Alight_SUB_CNC']
Passengers['Board_WARD_KE'] = boarding_join['Board_WARD_KE']
Passengers['Board_SUB_CNC'] = boarding_join['Board_SUB_CNC']
Passengers = Passengers.reset_index()

C:\Users\charl\AppData\Local\Continuum\anaconda3\envs\Cfox_2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\charl\AppData\Local\Continuum\anaconda3\envs\Cfox_2\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


### Passengers: Fare Summary Output Tables

In [8]:
ethnicity_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Ethnicity']).median()
age_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Age Group']).median()
gender_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Gender']).median()
gender_ethnicity_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Gender','Ethnicity']).median()
age_ethnicity_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Age Group','Ethnicity']).median()
full_table = Passengers[['Ethnicity', 'Age Group', 'Gender', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby(['Gender','Age Group','Ethnicity']).median()
tables = [ethnicity_table, age_table, gender_table, gender_ethnicity_table, age_ethnicity_table, full_table]

### Trip level analytics

In [9]:
unique_trips = list(set(Passengers['Trip Route Description']))
Selected_trips = unique_trips[:10]
trip_tables = []
for t in Selected_trips:
    Q = Passengers.loc[Passengers['Trip Route Description'].isin([t])]
    ethnicities = ['BLACK','INDIAN','OTHER','WHITE','COLOURED']
    genders = ['M','F']
    ages = ['YOUNG','MIDDLE','OLD']
    s = Q.copy()
    
    for gend in genders:
        s[gend] = 0
        s[gend].loc[s['Gender'] == gend] = 1
    for ethn in ethnicities:
        s[ethn] = 0
        s[ethn].loc[s['Ethnicity'] == ethn] = 1
    for age in ages:
        s[age] = 0
        s[age].loc[s['Age Group'] == age] = 1
    s['Total'] = s['M'] + s['F']
    col_keep = []
    s['Trip_st_hour'] = s['Trip Start Time'].apply(lambda x: x.hour)
    s = s.groupby('Trip_st_hour').sum()
    s = s.sort_values(by = 'Trip_st_hour', ascending = True)
    for ethn in ethnicities:
        s['% {}'.format(ethn)] = s[ethn] / (s['Total'])
        col_keep.append('% {}'.format(ethn))
    for gend in genders:
        s['% {}'.format(gend)] = s[gend] / (s['Total'])
        col_keep.append('% {}'.format(gend))
    for age in ages:
        s['% {}'.format(age)] = s[age] / (s['Total'])
        col_keep.append('% {}'.format(age)) 
    s = s[['M','F',*col_keep,]]
    trip_tables.append(s)

C:\Users\charl\AppData\Local\Continuum\anaconda3\envs\Cfox_2\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Route Summary: Initial Generation

In [10]:
import warnings
warnings.filterwarnings('ignore')

route_summary = Passengers[['Trip Route Description', 'Fare', 'Fare_per_minute', 'Fare_per_mile', 'TT_minutes']].groupby('Trip Route Description').median().sort_values(by = 'Fare_per_minute', ascending = False)
route_summary['Origin'] = RouteMaster['ORIGIN']
route_summary['Destination'] = RouteMaster['DESTINATION']

s = Passengers[['Trip Route Description', 'Gender','Ethnicity','Age Group']]
ethnicities = ['BLACK','INDIAN','OTHER','WHITE','COLOURED']
genders = ['M','F']
ages = ['YOUNG','MIDDLE','OLD']
for gend in genders:
    s[gend] = 0
    s[gend].loc[s['Gender'] == gend] = 1
for ethn in ethnicities:
    s[ethn] = 0
    s[ethn].loc[s['Ethnicity'] == ethn] = 1
for age in ages:
    s[age] = 0
    s[age].loc[s['Age Group'] == age] = 1
s = s.groupby('Trip Route Description').sum()
s['Total'] = s['M'] + s['F']
col_keep = []
for ethn in ethnicities:
    s['% {}'.format(ethn)] = s[ethn] / (s['Total'])
    col_keep.append('% {}'.format(ethn))
for gend in genders:
    s['% {}'.format(gend)] = s[gend] / (s['Total'])
    col_keep.append('% {}'.format(gend))
for age in ages:
    s['% {}'.format(age)] = s[age] / (s['Total'])
    col_keep.append('% {}'.format(age))
s = s[col_keep]
for i in col_keep:
    route_summary[i] = s[i]

### Route Summary: Match on Route WKT

In [11]:
import geopandas as gpd
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default

route_summary = route_summary.reset_index()
set_of_routes = list(set(route_summary['Trip Route Description']))

for root, folder, files in os.walk(r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Submission_15_12_2018\COCT_Dec'):
    pass

gathered = []
geom_dict = {}
for f in files:
    route = f.split('_')[0]
    if route in set_of_routes and route not in gathered:
        if f.split('_')[2] == 'trip.kml':
            f = gpd.read_file(os.path.join(root, f))
            geom_dict[route] = f.geometry.iloc[0]
            gathered.append(route)
            
geom_df = pd.DataFrame({'geometry':list(geom_dict.values())}, index = geom_dict.keys())
route_summary = route_summary.set_index('Trip Route Description')
route_summary['WKT'] = geom_df['geometry']

### Route Summary: Match on Start / End Ward

In [12]:
route_summary = route_summary.reset_index()
route_summary = route_summary.loc[route_summary.WKT.apply(type) != float]
route_summary['Origin_Point'] = route_summary.WKT.apply(lambda x: Point((x.coords[0])))
route_summary['Destination_Point'] = route_summary.WKT.apply(lambda x: Point((x.coords[-1])))

trip_origin_df = route_summary[['Trip Route Description','Origin_Point']]
trip_origin_df = gpd.GeoDataFrame(trip_origin_df, geometry = 'Origin_Point', crs = ward_shp.crs)
trip_origin_join = gpd.sjoin(trip_origin_df, ward_shp, how = 'left')
trip_origin_join = trip_origin_join.set_index('Trip Route Description')

trip_dest_df = route_summary[['Trip Route Description','Destination_Point']]
trip_dest_df = gpd.GeoDataFrame(trip_dest_df, geometry = 'Destination_Point', crs = ward_shp.crs)
trip_dest_join = gpd.sjoin(trip_dest_df, ward_shp, how = 'left')
trip_dest_join = trip_dest_join.set_index('Trip Route Description')

route_summary = route_summary.set_index('Trip Route Description')
route_summary['dest_ward_KE'] = trip_dest_join['SL_WARD_KE']
route_summary['dest_SUB_CNC'] = trip_dest_join['SL_SUB_CNC']
route_summary['origin_ward_KE'] = trip_origin_join['SL_WARD_KE']
route_summary['origin_SUB_CNC'] = trip_origin_join['SL_SUB_CNC']

### Time of Day: Generate Analytics

In [13]:
s = Passengers.copy()
s['Boarding Hour'] = s['Boarding Time'].apply(lambda x: x.hour)
col_keep = []
for gend in genders:
    s[gend] = 0
    s[gend].loc[s['Gender'] == gend] = 1
    col_keep.append(gend)
for ethn in ethnicities:
    s[ethn] = 0
    s[ethn].loc[s['Ethnicity'] == ethn] = 1
    col_keep.append(ethn)
for age in ages:
    s[age] = 0
    s[age].loc[s['Age Group'] == age] = 1
    col_keep.append(age)

In [14]:
data_averageing = s[['Boarding Hour','Fare','Fare_per_mile','Fare_per_minute','TT_minutes']]
data_counting = s[['Boarding Hour',*col_keep]]
data_averageing = data_averageing.groupby('Boarding Hour').median()
data_counting = data_counting.groupby('Boarding Hour').sum()
data = data_averageing.join(data_counting)
data.columns = ['Median Fare','Median Fare per Mile', 'Median Fare per Minute', 'Median Trip Time (minutes)', *col_keep]

In [15]:
s = data.copy()
s['Total'] = s['M'] + s['F']
for ethn in ethnicities:
    s['% {}'.format(ethn)] = s[ethn] / (s['Total'])
for gend in genders:
    s['% {}'.format(gend)] = s[gend] / (s['Total'])
for age in ages:
    s['% {}'.format(age)] = s[age] / (s['Total'])
time_of_day = s.copy()

### Write Out

In [16]:
with pd.ExcelWriter(os.path.join(p, 'summary.xlsx')) as writer:
    route_summary.to_excel(writer, sheet_name = 'RouteSummary')
    time_of_day.to_excel(writer, sheet_name = 'HourlyAnalysis')
    counter = 1
    for table in tables:
        table.to_excel(writer, sheet_name = 'table_%s' % counter)
        counter +=1
    counter = 1
    for table in trip_tables:
        table.to_excel(writer, sheet_name = '%s' % Selected_trips[counter - 1])
        counter +=1

### Analyis of Khayelitsha

In [101]:
p_s = r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Fwd__SAL'
khay = gpd.read_file(os.path.join(p_s, 'Khayelitsha.shp'))

khay_shp = unary_union(khay.geometry)
khay_shp_gdf = gpd.GeoDataFrame({'geometry':khay_shp}, crs = {'init':'epsg:4326'}, index = [1], geometry = 'geometry')
khay_shp_gdf.to_file(os.path.join(p_s, 'Khayelitsha_area.shp'), driver = 'ESRI Shapefile')
khay_shp_gdf = khay_shp_gdf.to_crs({'init':'epsg:22234'})
khay_shp_gdf.to_file(os.path.join(p_s, 'Khayelitsha_area_222234.shp'), driver = 'ESRI Shapefile')

shp_pth = r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Fwd__SAL\1_Residential_SAL'
ward_shp = gpd.read_file(os.path.join(shp_pth, 'wards_4326.shp'))
ward_shp = ward_shp.to_crs({'init':'epsg:22234'})
ward_shp.to_file(os.path.join(p_s, 'wards_222234.shp'), driver = 'ESRI Shapefile')

In [89]:
Passengers['Boarding Point'] = Passengers['Boarding Location'].apply(lambda x: convert_to_point(x))
Passengers['Alighting Point'] = Passengers['Alighting Location'].apply(lambda x: convert_to_point(x))

In [90]:
khay_p_boarding = gpd.GeoDataFrame(Passengers, crs = khay.crs, geometry = 'Boarding Point')
khay_p_boarding['Boarding Time hour'] = khay_p_boarding['Boarding Time'].apply(lambda x: x.hour)
khay_p_boarding = khay_p_boarding.loc[khay_p_boarding.intersects(khay_shp) == True]

In [91]:
khay_p_boarding = khay_p_boarding.to_crs({'init':'epsg:22234'})
khay_p_boarding.to_csv(os.path.join(p, 'khay.csv'))

In [92]:
khay_p_boarding_men = khay_p_boarding.loc[khay_p_boarding['Gender'] == 'M']
khay_p_boarding_men.to_csv(os.path.join(p, 'khay_boarding_men.csv'))
khay_p_boarding_women = khay_p_boarding.loc[khay_p_boarding['Gender'] == 'F']
khay_p_boarding_women.to_csv(os.path.join(p, 'khay_boarding_women.csv'))

In [93]:
khay_p_alighting = gpd.GeoDataFrame(Passengers, crs = khay.crs, geometry = 'Alighting Point')
khay_p_alighting['Alighting Time hour'] = khay_p_alighting['Alighting Time'].apply(lambda x: x.hour)
khay_p_alighting = khay_p_alighting.loc[khay_p_alighting.intersects(khay_shp) == True]

In [94]:
khay_p_alighting = khay_p_alighting.to_crs({'init':'epsg:22234'})
khay_p_alighting_men = khay_p_alighting.loc[khay_p_alighting['Gender'] == 'M']
khay_p_alighting_men.to_csv(os.path.join(p, 'khay_alighting_men.csv'))
khay_p_alighting_women = khay_p_alighting.loc[khay_p_alighting['Gender'] == 'F']
khay_p_alighting_women.to_csv(os.path.join(p, 'khay_alighting_women.csv'))

In [98]:
shp_pth = r'C:\Users\charl\Documents\GOST\South Africa\Connectivity\Fwd__SAL\1_Residential_SAL'
ward_shp = gpd.read_file(os.path.join(shp_pth, 'wards_4326.shp'))
ward_shp['centroid'] = ward_shp.geometry.centroid
match_ward = ward_shp[['SL_WARD_KE','centroid']].set_index('SL_WARD_KE')
match_ward['centroid'].loc[5177473] = Point(18.3918, -33.9201)

In [99]:
khayelitsha_wards = [5177561,5177559, 5177557, 5177555, 5177553, 5177551, 5177549, 5177547, 5177545, 5177543]
times = {'morning':[7,8,9],'midday':[11,12,13],'evening':[16,17,8,19]}
outs = []
for t in times:
    HRS = times[t]
    binz = []
    for ward in khayelitsha_wards:
        Q = khay_p_boarding.copy()
        Q = Q.loc[Q['Boarding Time hour'].isin(HRS)]
        q = Q.loc[Q.Board_WARD_KE == ward]
        dest = q.Alight_WARD_KE.value_counts().to_frame()
        dest = dest.reset_index()
        dest.columns = ['D_ID','count']
        dest['O_ID'] = ward
        binz.append(dest)
    df2 = pd.concat(binz)
    df2['time'] = str(t)
    outs.append(df2)

ins = []
for t in times:
    HRS = times[t]
    binz = []
    for ward in khayelitsha_wards:
        Q = khay_p_alighting.copy()
        Q = Q.loc[Q['Alighting Time hour'].isin(HRS)]
        q = Q.loc[Q.Alight_WARD_KE == ward]
        orig = q.Board_WARD_KE.value_counts().to_frame()
        orig = orig.reset_index()
        orig.columns = ['O_ID','count']
        orig['D_ID'] = ward
        binz.append(orig)
    df2 = pd.concat(binz)
    df2['time'] = str(t)
    ins.append(df2)

nets = []
for i in range(0, len(ins)):
    A = ins[i]
    A['combo'] = A['O_ID'].astype(str) + ' | ' + A['D_ID'].astype(str)
    A = A.rename({'count':'inflow'}, axis = 1)
    A = A[['inflow','combo']].set_index('combo')
    B = outs[i]
    B['combo'] = B['D_ID'].astype(str) + ' | ' + B['O_ID'].astype(str)
    B = B.rename({'count':'outflow'}, axis = 1)
    B = B[['outflow','combo']].set_index('combo')
    C = A.join(B, how = 'outer')
    C['inflow'] = C['inflow'].fillna(0)
    C['outflow'] = C['outflow'].fillna(0)
    C['net'] = C['inflow'] - C['outflow']
    C = C.sort_values(by = 'net', ascending = False)
    C['time'] = list(times.keys())[i]
    C = C.reset_index()
    C['O_ID'] = C['combo'].apply(lambda x: x.split(' | ')[0]).astype(float)
    C['D_ID'] = C['combo'].apply(lambda x: x.split(' | ')[1]).astype(float)
    nets.append(C)

for X in ['combined','disagg']:
    if X == 'disagg':
        for i in range(0, len(nets)):
            df2 = nets[i]
            time_of_day = list(times.keys())[i]
            df = df2.copy()
            df = df.set_index('D_ID')
            df['D_Point'] = match_ward['centroid']
            df = df.reset_index().set_index('O_ID')
            df['O_Point'] = match_ward['centroid']
            df = df.reset_index()
            df['journey'] = df['O_ID'].astype(str) + ' to ' + df['D_ID'].astype(str)
            df['WKT'] = df.apply(lambda x: LineString([x.O_Point,x.D_Point]), axis = 1)
            df.to_csv(os.path.join(p, 'Khayelitsha', '{}_{}.csv'.format(time_of_day, X)))
    elif X == 'combined':
        for i in range(0, len(nets)):
            time_of_day = list(times.keys())[i]
            df2 = nets[i]
            df2 = df2[['net','time','D_ID','O_ID']]
            df2 = df2.groupby('O_ID').sum().reset_index()
            df = df2.copy()
            df = df.set_index('D_ID')
            df['D_Point'] = khay_shp.centroid
            df = df.reset_index().set_index('O_ID')
            df['O_Point'] = match_ward['centroid']
            df = df.reset_index()
            df['journey'] = df['O_ID'].astype(str) + ' to ' + df['D_ID'].astype(str)
            df['WKT'] = df.apply(lambda x: LineString([x.O_Point,x.D_Point]), axis = 1)
            df.to_csv(os.path.join(p, 'Khayelitsha', '{}_{}.csv'.format(time_of_day, X)))